In [1]:
# Import libraries - REQUIRES pip version 9.0.3
import pandas
import os
from os.path import join
import sys
import copy

# Using Cobrapy 0.13.0
import cobra
import cobra.test
from cobra import Model
from cobra import Metabolite
from cobra import Reaction
from cobra.io import write_sbml_model

# Estabish handler for logger
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

# Verbose exception printing
%xmode

Exception reporting mode: Verbose


In [2]:
# Initialize model
toy = Model('toy_model')


In [3]:
# Define reactions and metabolites

cpdA_e = Metabolite(
    'cpdA_e',
    name='compound A',
    compartment='e')
cpdA_c = Metabolite(
    'cpdA_c',
    name='compound A',
    compartment='c')

rxn1 = Reaction('rxn1')
rxn1.name = 'Transport'
rxn1.gene_reaction_rule = 'gene1'
rxn1.lower_bound = 0. 
rxn1.upper_bound = 1000.  
rxn1.add_metabolites({
    cpdA_e: -1.0,
    cpdA_c: 1.0
})

cpdB_c = Metabolite(
    'cpdB_c',
    name='compound B',
    compartment='c')

rxn2 = Reaction('rxn2')
rxn2.name = 'Metabolite conversion 1'
rxn2.gene_reaction_rule = 'gene2'
rxn2.lower_bound = -1000. 
rxn2.upper_bound = 1000.  
rxn2.add_metabolites({
    cpdA_c: -1.0,
    cpdB_c: 1.0
})

cpdC_c = Metabolite(
    'cpdC_c',
    name='compound C',
    compartment='c')

rxn3 = Reaction('rxn3')
rxn3.name = 'Metabolite conversion 2'
rxn3.gene_reaction_rule = 'gene3'
rxn3.lower_bound = 0. 
rxn3.upper_bound = 1000.  
rxn3.add_metabolites({
    cpdB_c: -1.0,
    cpdC_c: 1.0
})

cpdD_c = Metabolite(
    'cpdD_c',
    name='compound D',
    compartment='c')

rxn4 = Reaction('rxn4')
rxn4.name = 'Metabolite conversion 3'
rxn4.gene_reaction_rule = 'gene4'
rxn4.lower_bound = 0. 
rxn4.upper_bound = 1000.  
rxn4.add_metabolites({
    cpdC_c: -1.0,
    cpdD_c: 1.0
})

cpdE_c = Metabolite(
    'cpdE_c',
    name='compound E',
    compartment='c')

rxn5 = Reaction('rxn5')
rxn5.name = 'Metabolite conversion 4'
rxn5.gene_reaction_rule = 'gene5'
rxn5.lower_bound = 0. 
rxn5.upper_bound = 1000.  
rxn5.add_metabolites({
    cpdC_c: -1.0,
    cpdE_c: 1.0
})

rxn6 = Reaction('rxn6')
rxn6.name = 'Metabolite conversion 5'
rxn6.lower_bound = 0. 
rxn6.upper_bound = 1000.  
rxn6.add_metabolites({
    cpdE_c: -1.0,
    cpdD_c: 1.0
})

biomass_cpd = Metabolite(
    'biomass_cpd',
    name='Biomass',
    compartment='c')

biomass_rxn = Reaction('biomass_rxn')
biomass_rxn.name = 'Biomass reaction'
biomass_rxn.lower_bound = 0. 
biomass_rxn.upper_bound = 1000.  
biomass_rxn.add_metabolites({
    cpdD_c: -1.0,
    biomass_cpd: 1.0
})

toy.add_reactions([rxn1,rxn2,rxn3,rxn4,rxn5,rxn6,biomass_rxn])



In [4]:
# Add input and output exchanges
toy.add_boundary(cpdA_e, type='exchange', reaction_id='EX_cpdA_e', lb=-1000.0, ub=1000.0)
toy.add_boundary(biomass_cpd, type='demand', reaction_id='EX_biomass', lb=None, ub=1000.0)


Reaction identifier,DM_biomass_cpd
Name,Biomass demand
Memory address,0x07fb0014b1c10
Stoichiometry,biomass_cpd --> Biomass -->
GPR,
Lower bound,0
Upper bound,1000.0


In [5]:
toy

Name,toy_model
Memory address,0x07fb040e24c50
Number of metabolites,7
Number of reactions,9
Objective expression,0
Compartments,"c, e"


In [6]:
#!/usr/bin/python
'''
Gapfilling function that utilizes pFBA and flux sampling to find most
parsimonious additional reactions to achieve minimum flux through the objective
Author: Matthew Jenior
'''
import pandas
import math
import copy
import time
import random

# Using Cobrapy 0.13.0
import cobra
import cobra.test
from cobra.flux_analysis.sampling import OptGPSampler
from cobra.manipulation.delete import *
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.medium import find_boundary_types

# pFBA gapfiller
def pfba_gapfill(model, reaction_bag, likelihoods, obj=None, obj_lb=10., obj_constraint=False,
                 iters=1, tasks=None, task_lb=0.05, 
                 add_exchanges=True, extracellular='e'):
    '''
    Function that utilizes iterations of pFBA solution with a universal reaction bag 
    in order to gapfill a model.
    
    Parameters
    ----------
    model : cobra.Model
        Model to be gapfilled
    reaction_bag : cobra.Model
        Reaction bag reference to use during gapfilling
    obj : string
        Reaction ID for objective function in model to be gapfilled.
    obj_lb : float
        Lower bound for objective function
    obj_constraint : bool
        Sets objective as contstraint which must be maximized
    tasks : list or None
        List of reactions IDs (strings) of metabolic tasks 
        to set a minimum lower bound for
    task_lb : float
        Lower bound for any metabolic tasks
    iters : int
        Number of gapfilling rounds. Unique reactions from each round are 
        saved and the union is added simulatneously to the model
    add_exchanges : bool
        Identifies extracellular metabolites added during gapfilling that
        are not associated with exchange reactions and creates them
    extracellular : string
        Label for extracellular compartment of model
    '''
    start_time = time.time()
    
    # Save some basic network info for downstream membership testing
    orig_rxn_ids = set([str(x.id) for x in model.reactions])
    orig_cpd_ids = set([str(y.id) for y in model.metabolites])
    univ_rxn_ids = set([str(z.id) for z in reaction_bag.reactions])
    
    # Find overlap in model and reaction bag
    overlap_rxn_ids = univ_rxn_ids.intersection(orig_rxn_ids)
    
    # Get model objective reaction ID
    if obj == None:
        obj = get_objective(model)
    else:
        obj = obj
    
    # Modify universal reaction bag
    new_rxn_ids = set()
    print('Creating universal model...')
    with reaction_bag as universal:

        # Remove overlapping reactions from universal bag, and reset objective if needed
        for rxn in overlap_rxn_ids: 
            universal.reactions.get_by_id(rxn).remove_from_model()
        
        # Set objective in universal if told by user
        # Made constraint as fraction of minimum in next step
        if obj_constraint == True:
            universal.add_reactions([model.reactions.get_by_id(obj)])
            universal.objective = obj
            orig_rxn_ids.remove(obj)
            orig_rxns = []
            for rxn in orig_rxn_ids: 
                orig_rxns.append(copy.deepcopy(model.reactions.get_by_id(rxn)))
        else:
            orig_rxns = list(copy.deepcopy(model.reactions))
            
        # Add pFBA to universal model and add model reactions
        add_pfba_likely(universal, likelihoods)
        
        updated_universal = copy.deepcopy(universal)
        universal = copy.deepcopy(universal) # reset solver
        universal.add_reactions(orig_rxns)
        
        # If previous objective not set as constraint, set minimum lower bound
        if obj_constraint == False: 
            universal.reactions.get_by_id(obj).lower_bound = obj_lb
    
        # Set metabolic tasks that must carry flux in gapfilled solution
        if tasks != None:
            for task in tasks:                    
                universal.reactions.get_by_id(task).lower_bound = task_lb
                
        # Run FBA and save solution
        print('Optimizing model with combined reactions...')
        solution = universal.optimize()

        if iters > 1:
            print('Generating flux sampling object...')
            optgp_object = OptGPSampler(universal, processes=4)
        
            # Assess the sampled flux distributions
            print('Sampling ' + str(iters) + ' flux distributions...')
            flux_samples = optgp_object.sample(iters)
            rxns = list(flux_samples.columns)
            for distribution in flux_samples.iterrows():
                for flux in range(0, len(list(distribution[1]))):
                    if abs(list(distribution[1])[flux]) > 1e-6:
                        new_rxn_ids |= set([rxns[flux]]).difference(orig_rxn_ids)
        else:
            rxns = list(solution.fluxes.index)
            fluxes = list(solution.fluxes)
            for flux in range(0, len(fluxes)):
                if abs(fluxes[flux]) > 1e-6:
                    new_rxn_ids |= set([rxns[flux]])
    
    # Screen new reaction IDs
    if obj in new_rxn_ids: new_rxn_ids.remove(obj)
    for rxn in orig_rxn_ids:
        try:
            new_rxn_ids.remove(rxn)
        except:
            continue
    
    # Get reactions and metabolites to be added to the model
    print('Retrieving reactions and metabolites needed for gapfilling...')
    new_rxns = copy.deepcopy([reaction_bag.reactions.get_by_id(rxn) for rxn in new_rxn_ids])
    new_cpd_ids = set()
    for rxn in new_rxns: new_cpd_ids |= set([str(x.id) for x in list(rxn.metabolites)])
    new_cpd_ids = new_cpd_ids.difference(orig_cpd_ids)
    new_cpds = copy.deepcopy([reaction_bag.metabolites.get_by_id(cpd) for cpd in new_cpd_ids])
    
    # Copy model and gapfill 
    print('Gapfilling model...')
    new_model = copy.deepcopy(model)
    new_model.add_metabolites(new_cpds)
    new_model.add_reactions(new_rxns)
    
    # Identify extracellular metabolites with no exchanges
    if add_exchanges == True:
        new_exchanges = extend_exchanges(new_model, new_cpd_ids, extracellular)
        if len(new_exchanges) > 0: new_rxn_ids |= new_exchanges
    
    duration = int(round(time.time() - start_time))
    print('Took ' + str(duration) + ' seconds to gapfill ' + str(len(new_rxn_ids)) + \
          ' reactions and ' + str(len(new_cpd_ids)) + ' metabolites.') 
    
    new_obj_val = new_model.slim_optimize()
    if new_obj_val > 1e-6:
        print('Gapfilled model objective now carries flux (' + str(new_obj_val) + ').')
    else:
        print('Gapfilled model objective still does not carry flux.')
    
    return new_model
#     return updated_universal

# Adds missing exchanges for extracellulart metbaolites
def extend_exchanges(model, cpd_ids, ex):
    
    model_exchanges = set(find_boundary_types(model, 'exchange', external_compartment=ex))
    new_ex_ids = set()
    
    for cpd in cpd_ids:
        cpd = model.metabolites.get_by_id(cpd)
        if str(cpd.compartment) != ex:
            continue
        else:
            if bool(set(cpd.reactions) & model_exchanges) == False:
                try:
                    new_id = 'EX_' + cpd.id
                    model.add_boundary(cpd, type='exchange', reaction_id=new_id, lb=-1000.0, ub=1000.0)
                    new_ex_ids |= set([new_id])
                except ValueError:
                    pass

    return new_ex_ids


# Returns the reaction ID of the objective reaction
def get_objective(model):
    
    if len(list(model.objective.variables)) == 0:
        raise IndexError('Model has no objective set.')
    
    expression = str(model.objective.expression).split()
    if 'reverse' in expression[0]:
        obj_id = expression[2].split('*')[-1]
    else:
        obj_id = expression[0].split('*')[-1]
            
    return obj_id

from __future__ import absolute_import

import logging
from warnings import warn
from itertools import chain

from optlang.symbolics import Zero

from cobra.util import solver as sutil
from cobra.core.solution import get_solution

LOGGER = logging.getLogger(__name__)

def add_pfba_likely(model, likelihoods, objective=None, fraction_of_optimum=1.0):
    """Add pFBA objective

    Add objective to minimize the summed flux of all reactions to the
    current objective.

    See Also
    -------
    pfba

    Parameters
    ----------
    model : cobra.Model
        The model to add the objective to
    objective :
        An objective to set in combination with the pFBA objective.
    fraction_of_optimum : float
        Fraction of optimum which must be maintained. The original objective
        reaction is constrained to be greater than maximal_value *
        fraction_of_optimum.
    """
    if objective is not None:
        model.objective = objective
    if model.solver.objective.name == '_pfba_objective':
        raise ValueError('The model already has a pFBA objective.')
    sutil.fix_objective_as_constraint(model, fraction=fraction_of_optimum)
    reaction_variables = ((rxn.forward_variable, rxn.reverse_variable)
                          for rxn in model.reactions)
    variables = chain(*reaction_variables)
#     print(variables)
    dict1 = {}
    fail_report = []
#     count = 0
    for v in variables:
#         count += 1
#         print(str(count))
        for rxn in model.reactions:
            if v.name.startswith(rxn.id):
#                 print('hit')
                try:
                    dict1[v] = max(0.0, 1.0 - likelihoods[rxn.id])
#                     print('likelihood set')
                except:
                    try:
                        dict1[v] = 1.0
                    except:
                        print('FAILED')
                        pass
                    pass
            else:
                fail_report.append(1)

    model.objective = model.problem.Objective(
        Zero, direction='min', sloppy=True, name="_pfba_objective")
    model.objective.set_linear_coefficients(dict1)
#     return dict1

In [7]:
toy_uni = copy.deepcopy(toy)
toy.remove_reactions([rxn4,rxn6])
likelihoods = {}

likelihoods['rxn4'] = 0.0

likelihoods['rxn6'] = 1.

toy.objective = 'biomass_rxn'


In [8]:
toy.reactions

[<Reaction rxn1 at 0x7fb00145cad0>,
 <Reaction rxn2 at 0x7fb00145cb10>,
 <Reaction rxn3 at 0x7fb00145cb90>,
 <Reaction rxn5 at 0x7fb00145cd10>,
 <Reaction biomass_rxn at 0x7fb00145ce90>,
 <Reaction EX_cpdA_e at 0x7fb0014b1b90>,
 <Reaction DM_biomass_cpd at 0x7fb0014b1c10>]

In [9]:
toy_uni.reactions

[<Reaction rxn1 at 0x7fb0014b1890>,
 <Reaction rxn2 at 0x7fb0014b13d0>,
 <Reaction rxn3 at 0x7fb0014b1250>,
 <Reaction rxn4 at 0x7fb0013d5550>,
 <Reaction rxn5 at 0x7fb0013d5850>,
 <Reaction rxn6 at 0x7fb0013d51d0>,
 <Reaction biomass_rxn at 0x7fb0013d5090>,
 <Reaction EX_cpdA_e at 0x7fb0013d5c90>,
 <Reaction DM_biomass_cpd at 0x7fb0013d5b10>]

In [10]:
# dict_out = add_pfba_likely(toy_uni, likelihoods, objective=None, fraction_of_optimum=1.0)
# dict_out

In [11]:
new_model = pfba_gapfill(toy, toy_uni, likelihoods, obj=None, obj_lb=10., obj_constraint=False, iters=1, add_exchanges=False)


Creating universal model...
Optimizing model with combined reactions...
Retrieving reactions and metabolites needed for gapfilling...
Gapfilling model...
Took 0 seconds to gapfill 1 reactions and 0 metabolites.
Gapfilled model objective now carries flux (1000.0).


In [12]:
new_model.reactions


[<Reaction rxn1 at 0x7fb001433cd0>,
 <Reaction rxn2 at 0x7fb001433e50>,
 <Reaction rxn3 at 0x7fb001433f10>,
 <Reaction rxn5 at 0x7fb001433fd0>,
 <Reaction biomass_rxn at 0x7fb0014400d0>,
 <Reaction EX_cpdA_e at 0x7fb0014401d0>,
 <Reaction DM_biomass_cpd at 0x7fb001440210>,
 <Reaction rxn6 at 0x7fb00146aa10>]

In [20]:
reaction_variables = ((rxn.forward_variable, rxn.reverse_variable)
                      for rxn in new_model.reactions)
variables = chain(*reaction_variables)
for v in variables:
    v_split = v.name.split('_')
    print(v_split[0])
    
#     for rxn in model.reactions:
#         if v.name.startswith(rxn.id):
            


rxn1
rxn1
rxn2
rxn2
rxn3
rxn3
rxn5
rxn5
biomass
biomass
EX
EX
DM
DM
rxn6
rxn6


In [13]:
# Write to an SBML for later use
# cobra.io.write_sbml_model(toy, 'data/toy_model.sbml')